# TensorBoard Scalars: Logging basic training metrics in Keras

A basic task in machine learning is understanding how key metrics such as loss and accuracy change as training progresses. These metrics can help you understand if you're [overfitting](https://en.wikipedia.org/wiki/Overfitting), for example, or if you're unnecessarily training for too long. You may also want to compare these metrics across different training runs to help debug and improve your model.

TensorFlow's Scalar Summary API allows you to visualize these metrics in TensorBoard with very little effort. This tutorial presents a very basic example to give you an introduction to using Scalar Summaries when developing your Keras model.


In [0]:
# Ensure TensorFlow 2.0 is installed.
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

Skipping tensorboard as it is not installed.


In [0]:
import tensorflow as tf
print("TF version: ", tf.__version__)

from datetime import datetime

import numpy as np
from tensorflow import keras

TF version:  2.0.0-dev20190221


First, start with a clean slate and clear any logs from previous runs.

In [0]:
!rm -rf logs/ 

## Setting up data for regression

You're now going to use Keras to calculate a regression, i.e., find the best line of fit for a paired data set. (While using a gradient descent and neural networks is [overkill for this kind of problem](https://https://stats.stackexchange.com/questions/160179/do-we-need-gradient-descent-to-find-the-coefficients-of-a-linear-regression-mode), it does make for a very easy to understand example.)

You're going to use TensorBoard to observe how training and test **loss** change across epochs. Hopefully, you'll see training and test loss decrease over time and remain steady.

First, generate 1000 data points roughly along the line *y = 0.5x + 2*. Split these data points into training and test sets. The neural net will learn this relationship.

In [0]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)

X = np.linspace(-1, 1, data_size)
np.random.shuffle(X)

# f(X) = 0.5X + 2 + noise
Y = 0.5 * X + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train data.
X_train, Y_train = X[:train_size], Y[:train_size]
X_test, Y_test = X[train_size:], Y[train_size:]

## Start TensorBoard

Now, start TensorBoard. Wait a few seconds for TensorBoard's UI to spin up.

You'll see it say that "No dashboards are active for the current data set". That's because you haven't
logged any data yet in this session. Once you begin training, your Keras model will start logging data. TensorBoard will automatically refresh and show you your scalar metrics.

In [0]:
%tensorboard --logdir logs/scalars

To log the loss scalar, create the Keras TensorBoard callback, specifying the log directory as a timestamped subdirectory to allow easy identification and selection of training runs.

Pass the TensorBoard callback to [Model.fit()](https://https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit).

Once you've started training, scroll back up to TensorBoard and watch as the scalar graphs update.

You can zoom in and out of the graphs with your mouse, or select part of them to view more detail. Notice that both training and validation loss curves are overlaid for easy comparison. Things appear to have progressed well during training. In fact, you could have stopped training after 25 epochs or steps, because the training didn't improve after that point. 


In [ ]:
logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(2048, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(lr=0.2),
)

print("Training ... This takes less than 10 seconds.")
training_history = model.fit(
    X_train, # input
    Y_train, # output
    batch_size=train_size,
    verbose=0, # Suppress chatty output; we're going to look at Tensorboard
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

Training ... This takes less than 10 seconds.
Average test loss:  0.04592571808258072


## Try out the model

Ok, the metrics look good! Now see how the model actually behaves in real life. 

Given the input data (60, 25, 2), the line *y = 0.5x + 2* should yield (32, 14.5, 3). Does the model agree?

In [0]:
print(model.predict([60, 25, 2]))

[[31.967478 ]
 [14.485809 ]
 [ 2.9978569]]


Not bad!